# 网格搜索

<font color=red>网格搜索同样是交叉验证确定最优参数</font>

In [ ]:
def param_grid_search(train):
    """
    网格搜索参数寻优
    :param train:训练集
    :return:最优的分类器模型
    """
    print('param_grid_search')
    features = train.columns.tolist()
    features.remove("card_id")
    features.remove("target")
    
    parameter_space = {
        "n_estimators": [80,160],
        "min_samples_leaf": [30],
        "min_samples_split": [60,120],#
        "max_depth": [3,5,7,9],
        "max_features": ['sqrt','log2',50,100]
    }

    print("Tuning hyper-parameters for mse")
    clf = RandomForestRegressor(
        criterion="squared_error",
        min_weight_fraction_leaf=0.,#类似于min_samples_leaf，不过这里按照权重而不是数目
        max_leaf_nodes=None,
        min_impurity_decrease=0., #设置最低不纯度，低于这个值就不再分叉了
        bootstrap=True,
        oob_score=False,
        n_jobs=8,
        random_state=2020,
        verbose=0,
        warm_start=False)
    grid = GridSearchCV(clf, parameter_space, cv=2, scoring="neg_mean_squared_error")
    grid.fit(train[features].values, train['target'].values)

    print("best_params_:")
    print(grid.best_params_)
    print('====================================================================')
    means = grid.cv_results_["mean_test_score"]
    stds = grid.cv_results_["std_test_score"]
    i = 1
    for mean, std, params in zip(means, stds, grid.cv_results_["params"]):
        print(f'第{i}组参数:')
        i+=1
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return grid.best_estimator_
